In [ ]:
!which chromedriver

In [1]:
#import pymongo
from bs4 import BeautifulSoup as bs
import os
import requests
import shutil
import pandas as pd
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import urllib.request
import urllib.parse
import re
import time

#Dictionary to collect data
mars = {}

In [2]:
def init_browser():
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

### NASA Mars News

In [ ]:
browser = init_browser()

# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')
    #print(soup.prettify)

In [ ]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list

# Put together link to top story
link = [a['href'] for a in soup.select("a[href*=news]")][0]
base = "https://mars.nasa.gov/news"
url = urllib.parse.urljoin(base,link)

# Fetch headline and text  
browser.visit(url)
news_title = browser.find_by_tag('h1').value
news_p = browser.find_by_id('primary_column').value
news_p = news_p.replace('\n', ' ')
#tried a million ways to get rid of remaining \ (see old notebook), no luck yet
#But they are gone in the Chrome versio, thanks to whoever...
#story = {'news_title': news_title,'news_p': news_p}

# Add title and text to data dictionary 
mars["news_title"] = news_title
mars["news_p"] = news_p
  
news_title
news_p

### JPL Mars Space Images - Featured Image

In [ ]:
# Thanks to GitHub for the fancybox help, I was really stuck. Didn't realize "fancybox" could be a class. 
#https://github.com/ZL14E161110/HW13---Web-Scraping-and-Document-Databases
#Didn't look at the rest of it, although I'm sure it's brilliant. 

#Assumes the latest Mars image will always be the one in the first position of the grid...
#Note 0 is the overall featured image, which is not necessarily of Mars.

# Visit site and make soup
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
response = requests.get(url)
soup = bs(response.text, 'lxml')

#Get image 
partial_address = soup.find_all('a', class_='fancybox')[1].get('data-fancybox-href').strip()
base = "https://www.jpl.nasa.gov"
featured_image_url = urllib.parse.urljoin(base, partial_address)
#featured_image_url

#Add url to dictionary
mars["featured_image_url"] = featured_image_url

mars

### Mars Facts

In [ ]:
url = 'https://space-facts.com/mars/'

In [ ]:
tables = pd.read_html(url)
#tables = pd.read_html(url, header=None, index_col=0)
type(tables)

In [ ]:
mars_facts_df = tables[0]
mars_facts_df

In [ ]:
mars_facts_html_table = mars_facts_df.to_html(header=False, border=False, index=False)
mars_facts_html_table = mars_facts_html_table.replace('\n', '')
mars_facts_html_table
#print(mars_facts_html_table)

In [ ]:
#Add table to dictionary
mars["table"] = mars_facts_html_table
mars

### Mars Hemispheres

In [3]:
#navigate to starting page
browser=init_browser()
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [4]:
#Make soup
# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>
<html lang="en">
<head>
<link href="//ajax.googleapis.com/ajax/libs/jqueryui/1.11.3/themes/smoothness/jquery-ui.css" rel="stylesheet" type="text/css"/>
<title>Astropedia Search Results | USGS Astrogeology Science Center</title>
<meta content="USGS Astrogeology Science Center Astropedia search results." name="description"/>
<meta content="USGS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<meta content="x61hXXVj7wtfBSNOPnTftajMsZ5yB2W-qRoyr7GtOKM" name="google-site-verification"/>
<!--<link rel="stylesheet" href="http://fonts.googleapis.com/css?family=Open+Sans:400italic,400,bold"/>-->
<link href="/css/main.css" media="screen" rel="stylesheet"/>
<link href="/css/print.css" media="pr

In [ ]:
#Find titles and links for full-scale images

#OLD PATH TO 

#Find titles and links for full-scale images

#title_list = []
#img_url_list = []
base = "https://astrogeology.usgs.gov"
hemisphere_image_urls = []


for link in soup.find_all(class_='itemLink product-item'):
    partial = link.get('href')    
    thumbnail_url = urllib.parse.urljoin(base, partial)
    browser.visit(thumbnail_url)
    xpath = "//a[@class='open-toggle']"
    switch = browser.find_by_xpath(xpath)
    switch.click()
    html = browser.html
    soup = bs(html, 'lxml')
#    print(soup.prettify)    

    
    
#    imgs = soup.find(class_='downloads')
    html = browser.html
    soup = bs(html, 'lxml')
    title = soup.find(class_ = 'title').get_text()
    img_url = [a['href'] for a in soup.select('a:contains(Original)')]
    img_url = str(*img_url)
    hemisphere_image_urls.append({'title':title, 'img_url':img_url})
browser.quit()
hemisphere_image_urls

#mars["hemisphere_image_urls"] = hemisphere_image_urls
#mars






#    images = soup.a.img['src']    
#    print(images)
    
#    response = http.request('GET', url)
#    soup = BSHTML(response.data, "html.parser")
    
#    urls = soup.find_all('img',{'class':"wide-image"}) 
#    urls = soup.find_all('img',"wide-image") 
#    print(urls)
#    urls = soup.select('img[class="wide-image"]')
#    for url in urls:
        #print image source
#        print(url)        
#        print(image['src'])
        #print alternate text
        #print(image['alt'])    
    
    

    
#    all_images = soup.findAll('img')

#    for image in all_images:
        #print image source
#        print(image['src'])
        #print alternate text
#        print(image['alt'])
    
    
#    images = soup.findAll('img')

#    for image in images:
#        print (image['src'])
#    title = soup.find(class_ = 'title').get_text()
#    partial_address = soup.select('img', 'wide-image', 'src')
#    thumbnail_url = urllib.parse.urljoin(base, partial_address)
#    img_url = [a['href'] for a in soup.select('a:contains(Original)')]
#    img_url = str(*img_url)
#    thumbnail_url
#    hemisphere_image_urls.append({'title':title, 'img_url':img_url})
#browser.quit()
#hemisphere_image_urls

#mars["hemisphere_image_urls"] = hemisphere_image_urls
#mars

#print(image_url_list)


#    title_list.append(title)
#    image_url_list = img_url_list.append(*img_url)
#    title_list
#    print(title, *img_url)

In [5]:
#OLD PATH TO 

url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
time.sleep(10) 
# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')

#Find titles and links for full-scale images

base = "https://astrogeology.usgs.gov"
hemisphere_image_urls = []

for link in soup.find_all(class_='itemLink product-item'):
    partial = link.get('href')    
    thumbnail_url = urllib.parse.urljoin(base, partial)
    browser.visit(thumbnail_url)
    xpath = "//a[@class='open-toggle']"
    switch = browser.find_by_xpath(xpath)
    switch.click()
    html = browser.html
    soup = bs(html, 'lxml')
    imgs = soup.find(class_='downloads')
    html = browser.html
    soup = bs(html, 'lxml')
    title = soup.find(class_ = 'title').get_text()
    img_url = soup.find_all('img',{'class':"wide-image"}) 
#    images = soup.a.img['src']        
#    img_url = [a['href'] for a in soup.select('a:contains(Original)')]
    img_url = str(*img_url)
    hemisphere_image_urls.append({'title':title, 'img_url':img_url})
browser.quit()
hemisphere_image_urls
#    mars["hemisphere_image_urls"] = hemisphere_image_urls

#    return mars

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': '<img class="wide-image" src="/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg"/>'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': '<img class="wide-image" src="/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg"/>'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': '<img class="wide-image" src="/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg"/>'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': '<img class="wide-image" src="/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg"/>'}]

## Step 2 - MongoDB and Flask Application

Use MongoDB with Flask templating to create a new HTML page that displays all of the information that was scraped from the URLs above.

* Start by converting your Jupyter notebook into a Python script called `scrape_mars.py` with a function called `scrape` that will execute all of your scraping code from above and return one Python dictionary containing all of the scraped data.

* Next, create a route called `/scrape` that will call your `scrape_mars.scrape` function. Note that you'll have to import `scrape_mars.py`. 

  * Store the dictionary that gets returned to your MongoDB.

* Create an index route `/` that will query your Mongo database and pass the Mars data into an HTML template to be displayed. 

* Create a template HTML file called `index.html` that will take the dictionary of Mars data and display its values in the appropriate HTML elements. Use the following as a guide for what the final product should look like, but feel free to create your own design.

![final_app_part1.png](Images/final_app_part1.png)
![final_app_part2.png](Images/final_app_part2.png)